In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # display all the columns
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import datetime
from IPython.display import Image
from tqdm import tnrange, tqdm_notebook

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score


randSeed = 42

In [2]:
dataset = pd.read_csv("/Users/ps/Desktop/20191008hrupenndatasetshareable.csv")

In [3]:
dataset.shape

(19371, 93)

In [4]:
dataset['EMPLOYMENT_STATUS_DESC'].value_counts()

Active                          10371
Seperated or teminated           8985
Leave of absence without pay       15
Name: EMPLOYMENT_STATUS_DESC, dtype: int64

## Birth_Date

In [5]:

dataset["Birth_Date"] = pd.DatetimeIndex(dataset["Birth_Date"])

print(dataset["Birth_Date"])

0       1982-07-13
1       1984-02-10
2       1965-03-26
3       1959-09-30
4       1990-02-12
           ...    
19366   1963-07-05
19367   1991-12-06
19368   1990-12-17
19369   1974-04-19
19370   1984-04-15
Name: Birth_Date, Length: 19371, dtype: datetime64[ns]


In [6]:
types = []
months = []
days = []
years = []

for date in dataset["Birth_Date"]:
    types.append(type(date))
    months.append(date.month)
    days.append(date.day)
    years.append(date.year)

typesdf = pd.DataFrame(types)
monthsdf = pd.DataFrame(months)
daysdf = pd.DataFrame(days)
yearsdf = pd.DataFrame(years)


In [7]:
print("Months: Value Counts-")
print(monthsdf[0].value_counts())
print("Months: Unique Values-")
print(np.sort(monthsdf[0].unique()))
print("Years: Value Counts-")
print(yearsdf[0].value_counts())
print("Years: Unique Values-")
print(np.sort(yearsdf[0].unique()))
print("Days: Value Counts-")
print(daysdf[0].value_counts())
print("Days: Unique Values-")
print(np.sort(daysdf[0].unique()))


Months: Value Counts-
5     1741
8     1710
10    1673
7     1666
3     1642
9     1631
1     1626
12    1614
6     1576
11    1525
4     1522
2     1445
Name: 0, dtype: int64
Months: Unique Values-
[ 1  2  3  4  5  6  7  8  9 10 11 12]
Years: Value Counts-
1987    739
1986    709
1988    692
1984    688
1985    674
       ... 
1924      4
1926      4
1922      2
1914      1
1920      1
Name: 0, Length: 81, dtype: int64
Years: Unique Values-
[1914 1920 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933
 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947
 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961
 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975
 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000]
Days: Value Counts-
11    686
8     681
15    680
1     669
20    669
4     659
2     657
12    657
6     654
27    652
18    641
23    639
19

## RACE

In [8]:
def check_for_discrepency(df):
    df = df.drop_duplicates().dropna()
    weird = pd.DataFrame()
    for col in df.columns:
        weird = pd.concat([weird,df[df[col].duplicated()]], axis=0)
    if weird.size > 0:
        print('Something is wrong')
    else:
        print('All good')
    return weird

In [9]:
df1 = dataset[['RACE','Race_Desc']]
check_for_discrepency(df1)

All good


,RACE,Race_Desc


In [18]:
print(dataset["RACE"].value_counts())
print(np.sort(dataset["RACE"].unique()))
dataset[dataset["RACE"] == "B"]

C    12243
B     3575
R     2176
Z      947
S      256
U      105
A       44
P       25
Name: RACE, dtype: int64
['A' 'B' 'C' 'P' 'R' 'S' 'U' 'Z']


,Worker_ID,Birth_Date,RACE,Race_Desc,Race_OfColor,SEX,Sex_Desc,MARITAL_STATUS,AGE_AT_TERMINATION,CURRENT_AGE,EDUCATION_LEVEL,EDUCATION_LEVEL_DESC,EDUCATION_LEVEL_YEAR,COUNTRY_OF_CITIZENSHIP,HOME_ZIP,HOME_CITY,VISA,VISA_EXPIRATION_DATE,PRIMARY_APPT_BEGIN_DATE,YEARS_IN_PRIMARY_JOB,MONTHS_IN_PRIMARY_JOB,YEARS_OF_SERVICE,EMPLOYMENT_STATUS_DESC,HOME_DEPT_ORG,Dept_Name,Dept_Name_Short,HOME_SCHOOL_CTR,School_Name,School_Abbrev,ORIGINAL_HIRE_DATE,PRIMARY_APPT_JOB_CLASS,PRIMARY_APPT_SALARY,JOB_GRADE,JOB_TITLE,IN_UNION,PAY_CYCLE,GRANT_PAID,Job_Grade_Fiscal_Year,AnnualMinimum,AnnualFirstThird,AnnualMidPoint,AnnualSecondThird,AnnualMax,Starting_Full_Time_Salary,Career_Salary_Growth,Percent_Career_Salary_Growth,Number_of_Sal_Adjustments,Number_of_Reclass,REPORTS_TO_PENN_ID,REPORTS_TO_DATE,Manager_Years_Managed,Manager_Months_Managed,Manager_Original_Hire_Date,Manager_Current_Age,Manager_Age_At_Termination,Manager_Edu_Level,Manager_Edu_Level_Desc,Manager_Edu_Level_Year,Manager_YEARS_OF_SERVICE,Manager_Sex_Desc,Manager_Sex,Manager_Race_Desc,Manager_OfColor,Manager_Race,Manager_Job_Title,Termination_Fiscal_Year,TERMINATION_DATE,EMPLOYMENT_STATUS,SEPARATION_REASON,SEPERATION_REASON_DESC,VOLUNTARY,JOBS_ACTIVE,Retirement_Eligible_Date,OVR_RATING_CD_2010,OVR_RATING_DESCR_2010,OVR_RATING_CD_2011,OVR_RATING_DESCR_2011,OVR_RATING_CD_2012,OVR_RATING_DESCR_2012,OVR_RATING_CD_2013,OVR_RATING_DESCR_2013,OVR_RATING_CD_2014,OVR_RATING_DESCR_2014,OVR_RATING_CD_2015,OVR_RATING_DESCR_2015,OVR_RATING_CD_2016,OVR_RATING_DESCR_2016,OVR_RATING_CD_2017,OVR_RATING_DESCR_2017,OVR_RATING_CD_2018,OVR_RATING_DESCR_2018,OVR_RATING_CD_2019,OVR_RATING_DESCR_2019
9,10,1972-01-10,B,African American,1,M,Male,S,41.0,47,H,Master's Degree,2004,,19131,PHILADELPHIA,,NaN,11/16/2009,3,40,3,Seperated or teminated,603,Family and Community Health,FAM COM HEALTH,6,School of Nursing,Nursing,11/16/2009,120859,54695.56,28,MGR RES PROJECT B,0,M1,P,2013.0,52225.0,67893.0,75727.0,83560.0,99228.0,50000.0,4695.56,0.0939,0,0,10017259,3/4/2011,1.0,24.0,4/1/1995,64.0,57.0,K,Doctor of Philosophy (Ph.D.),1987,17.0,Female,F,African American,1.0,B,PROFESSOR A,2013.0,3/1/2013 0:00,T,41,Accepted New Job (Non-Academic),1.0,0,00:00.0,X,No Review,E,Exceeds,E,Exceeds,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review
10,11,1986-11-07,B,African American,1,M,Male,S,26.0,32,,NaN,,,19139,PHILADELPHIA,,NaN,4/2/2012,1,16,1,Seperated or teminated,603,Family and Community Health,FAM COM HEALTH,6,School of Nursing,Nursing,4/2/2012,525620,30525.00,23,ADMIN ASST A,0,W1,F,2014.0,24277.0,29941.0,32774.0,35606.0,41270.0,30000.0,525.00,0.0175,0,0,10017259,4/12/2013,0.0,4.0,4/1/1995,64.0,57.0,K,Doctor of Philosophy (Ph.D.),1987,18.0,Female,F,African American,1.0,B,PROFESSOR A,2014.0,8/9/2013 0:00,T,44,Personal Reasons,1.0,0,00:00.0,X,No Review,X,No Review,M,Meets,N,"Meets some, but not all",X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review
54,55,1968-06-12,B,African American,1,M,Male,S,NaN,51,K,Doctor of Philosophy (Ph.D.),1999,,19143-3711,PHILADELPHIA,,NaN,9/1/2005,14,169,17,Active,103,Biology,BIOLOGY,2,School of Arts and Sciences,SAS,7/11/2002,120859,99837.54,28,MGR RES PROJECT B,0,M1,N,2019.0,58527.0,76085.0,84864.0,93643.0,111201.0,42442.0,57395.54,1.3523,0,0,10218151,12/3/2018,0.0,9.0,7/1/2002,57.0,NaN,K,Doctor of Philosophy (Ph.D.),1990,17.0,Male,M,Asian,1.0,R,PROFESSOR A,NaN,NaN,A,,NaN,NaN,1,00:00.0,X,Appraisal not required,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds,X,No Review
80,81,1982-04-20,B,African American,1,M,Male,S,34.0,37,H,Master's Degree,2011,,19104-2916,PHILADELPHIA,,NaN,6/20/2011,5,67,5,Seperated or teminated,103,Biology,BIOLOGY,2,School of Arts and Sciences,SAS,6/20/2011,340181,73574.71,B,APPLICATION DEVELOPER B,0,M1,F,2017.0,40471.0,62608.0,73676.0,84744.0,106881.0,67000.0,6574.71,0.0981,0,0,10033240,4/7/2016,0.0,9.0,9/1/1994,59.0,57.0,K,Doctor of Philosophy (Ph.D.),1988,22.0,Male,M,Caucasian,0.0,C,IT TECHNI

## Race_OfColor

In [12]:
print(dataset["Race_OfColor"].value_counts())

0    12243
1     7128
Name: Race_OfColor, dtype: int64


In [15]:
print(np.sort(dataset["Race_OfColor"].unique()))

[0 1]


In [30]:
good = True
for i in range(len(dataset["Race_OfColor"])):
    if (dataset["Race_OfColor"][i] == 1) and (dataset["RACE"][i] == "C"):
        good = False
        
print(good)

True


## SEX

In [36]:
print(np.sort(dataset["SEX"].unique()))

['F' 'M']


In [37]:
print(dataset["SEX"].value_counts())

F    12114
M     7257
Name: SEX, dtype: int64


## MARITAL_STATUS

In [39]:
print(np.sort(dataset["MARITAL_STATUS"].unique()))

['D' 'M' 'S' 'W']


In [40]:
print(dataset["MARITAL_STATUS"].value_counts())

S    11568
M     7214
D      504
W       85
Name: MARITAL_STATUS, dtype: int64


## AGE_AT_TERMINATION

In [42]:
print(np.sort(dataset["AGE_AT_TERMINATION"].unique()))

[19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71. 72.
 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. 88. 89. 90.
 91. 92. 93. 94. 98. nan]


In [48]:
print(dataset["AGE_AT_TERMINATION"].value_counts().head())

24.0    658
25.0    592
26.0    522
27.0    507
23.0    462
Name: AGE_AT_TERMINATION, dtype: int64


In [47]:
print(dataset["AGE_AT_TERMINATION"].value_counts().tail())

19.0    3
93.0    2
98.0    1
94.0    1
90.0    1
Name: AGE_AT_TERMINATION, dtype: int64


In [54]:
def percent_null(series):
    # Calculates percentage of NaN values in DataFrame
 
    num = series.isnull().sum()
    den = len(series)
    return round(num/den, 2)

percent_null(dataset["AGE_AT_TERMINATION"])

0.54

## EDUCATION_LEVEL

In [49]:
print(np.sort(dataset["EDUCATION_LEVEL"].unique()))

['  ' 'B ' 'C ' 'D ' 'E ' 'F ' 'G ' 'H ' 'I ' 'J ' 'K ' 'L ']


In [53]:
print(dataset["EDUCATION_LEVEL"].value_counts().head())

G     7287
H     3956
      2833
C     1525
E     1317
Name: EDUCATION_LEVEL, dtype: int64


In [56]:
def percent_blank(series):
    # Calculates percentage of NaN values in DataFrame
    num = 0
    for element in series:
        if element == "  ":
            num += 1
    den = len(series)
    return round(num/den, 2)

percent_blank(dataset["EDUCATION_LEVEL"])

0.15

## EDUCATION_LEVEL_YEAR

In [57]:
print(np.sort(dataset["EDUCATION_LEVEL_YEAR"].unique()))

['    ' '1812' '1887' '1912' '1913' '1916' '1940' '1944' '1946' '1947'
 '1950' '1951' '1952' '1953' '1954' '1955' '1956' '1957' '1958' '1959'
 '1960' '1961' '1962' '1963' '1964' '1965' '1966' '1967' '1968' '1969'
 '1970' '1971' '1972' '1973' '1974' '1975' '1976' '1977' '1978' '1979'
 '1980' '1981' '1982' '1983' '1984' '1985' '1986' '1987' '1988' '1989'
 '1990' '1991' '1992' '1993' '1994' '1995' '1996' '1997' '1998' '1999'
 '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007' '2008' '2009'
 '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019'
 '2020' '2021' '2022' '2105' '2106']


In [70]:
print(dataset["EDUCATION_LEVEL_YEAR"].value_counts().head())

        5422
2010     727
2009     683
2012     654
2011     651
Name: EDUCATION_LEVEL_YEAR, dtype: int64


In [63]:
print(dataset["EDUCATION_LEVEL_YEAR"].value_counts().tail())

1916    1
1956    1
2105    1
2020    1
1887    1
Name: EDUCATION_LEVEL_YEAR, dtype: int64


In [60]:
def percent_blank2(series):
    # Calculates percentage of NaN values in DataFrame
    num = 0
    for element in series:
        if element == "    ":
            num += 1
    den = len(series)
    return round(num/den, 2)

percent_blank2(dataset["EDUCATION_LEVEL_YEAR"])

0.28

## PRIMARY_APPT_BEGIN_DATE

In [65]:
dataset["PRIMARY_APPT_BEGIN_DATE"] = pd.DatetimeIndex(dataset["PRIMARY_APPT_BEGIN_DATE"])

In [66]:
print(dataset["PRIMARY_APPT_BEGIN_DATE"])

0       2008-09-08
1       2011-01-03
2       2004-12-03
3       2002-07-01
4       2013-01-01
           ...    
19366   1998-05-11
19367   2014-06-15
19368   2013-07-30
19369   2002-08-19
19370   2009-11-11
Name: PRIMARY_APPT_BEGIN_DATE, Length: 19371, dtype: datetime64[ns]


In [67]:
types = []
months = []
days = []
years = []

for date in dataset["PRIMARY_APPT_BEGIN_DATE"]:
    types.append(type(date))
    months.append(date.month)
    days.append(date.day)
    years.append(date.year)

typesdf = pd.DataFrame(types)
monthsdf = pd.DataFrame(months)
daysdf = pd.DataFrame(days)
yearsdf = pd.DataFrame(years)

In [68]:
print("Months: Value Counts-")
print(monthsdf[0].value_counts())
print("Months: Unique Values-")
print(np.sort(monthsdf[0].unique()))
print("Years: Value Counts-")
print(yearsdf[0].value_counts())
print("Years: Unique Values-")
print(np.sort(yearsdf[0].unique()))
print("Days: Value Counts-")
print(daysdf[0].value_counts())
print("Days: Unique Values-")
print(np.sort(daysdf[0].unique()))


Months: Value Counts-
7     2533
9     1958
8     1896
6     1864
1     1723
10    1690
11    1461
5     1363
4     1346
3     1338
2     1304
12     895
Name: 0, dtype: int64
Months: Unique Values-
[ 1  2  3  4  5  6  7  8  9 10 11 12]
Years: Value Counts-
2018    2180
2017    1880
2016    1631
2015    1348
2014    1113
2019    1107
2011    1033
2013    1010
2012     995
2010     903
2008     890
2009     851
2007     715
2006     503
2005     387
2002     300
2003     273
2004     269
2001     256
2000     235
1999     172
1998     152
1997     136
1989      83
1996      82
1990      77
1991      70
1986      67
1994      66
1988      63
1992      61
1993      59
1995      57
1987      56
1985      41
1983      21
1984      20
1980      20
1982      19
1975      19
1981      18
1973      17
1979      14
1969      13
1978      12
1971      11
1974      10
1970       9
1972       9
1976       9
1968       8
1977       8
1966       5
1957       2
1967       2
1956       1
1962       1
1

## MONTHS_IN_PRIMARY_JOB

In [69]:
print(np.sort(dataset["MONTHS_IN_PRIMARY_JOB"].unique()))

[-12 -11  -4  -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13
  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31
  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49
  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175
 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193
 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211
 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229
 230 231 232 233 234 235 236 237 238 239 240 241 24

In [71]:
print(dataset["MONTHS_IN_PRIMARY_JOB"].value_counts().head())

9     430
15    427
11    421
12    401
4     394
Name: MONTHS_IN_PRIMARY_JOB, dtype: int64


## HOME_DEPT_ORG

In [78]:
print(np.sort(dataset["HOME_DEPT_ORG"].unique()))

[ 101  103  104  105  106  107  109  110  111  112  113  114  115  116
  117  118  119  120  121  122  124  125  126  127  129  141  145  146
  147  150  151  152  153  155  156  163  164  166  178  181  182  193
  201  210  218  222  225  227  228  232  233  234  239  242  244  247
  249  262  267  268  293  294  297  298  299  300  301  302  303  304
  305  310  311  312  314  316  326  327  328  331  336  400  401  406
  408  409  411  412  417  424  431  437  438  439  440  441  444  445
  446  602  603  608  619  620  623  624  625  626  631  632  640  641
  644  645  646  648  652  701  702  703  704  705  706  707  708  709
  710  711  712  713  714  719  721  722  724  725  726  727  729  730
  731  733  734  737  738  744  745  746  747  748  754  760  761  764
  766  767  768  771  777  779  780  783  784  789  790  791  792  801
  802  803  806  807  817  819  821  823  829  830  831  833  834  835
  837  838 1300 1301 1302 1303 1304 1305 1306 1308 1309 1311 1312 1313
 1316 

In [79]:
len(np.sort(dataset["HOME_DEPT_ORG"].unique()))

719

In [81]:
print(dataset["HOME_DEPT_ORG"].value_counts().head())

9603    759
8306    383
4607    345
4623    321
9604    319
Name: HOME_DEPT_ORG, dtype: int64


In [82]:
print(dataset["HOME_DEPT_ORG"].value_counts().tail())

4000    1
146     1
8342    1
249     1
141     1
Name: HOME_DEPT_ORG, dtype: int64


In [83]:
df2 = dataset[['HOME_DEPT_ORG','Dept_Name']]
check_for_discrepency(df2)

Something is wrong


,HOME_DEPT_ORG,Dept_Name
9021,5172,Online Learning
11401,8317,Arthur Ross Gallery


In [85]:
dataset[dataset["HOME_DEPT_ORG"] == 5172]

,Worker_ID,Birth_Date,RACE,Race_Desc,Race_OfColor,SEX,Sex_Desc,MARITAL_STATUS,AGE_AT_TERMINATION,CURRENT_AGE,EDUCATION_LEVEL,EDUCATION_LEVEL_DESC,EDUCATION_LEVEL_YEAR,COUNTRY_OF_CITIZENSHIP,HOME_ZIP,HOME_CITY,VISA,VISA_EXPIRATION_DATE,PRIMARY_APPT_BEGIN_DATE,YEARS_IN_PRIMARY_JOB,MONTHS_IN_PRIMARY_JOB,YEARS_OF_SERVICE,EMPLOYMENT_STATUS_DESC,HOME_DEPT_ORG,Dept_Name,Dept_Name_Short,HOME_SCHOOL_CTR,School_Name,School_Abbrev,ORIGINAL_HIRE_DATE,PRIMARY_APPT_JOB_CLASS,PRIMARY_APPT_SALARY,JOB_GRADE,JOB_TITLE,IN_UNION,PAY_CYCLE,GRANT_PAID,Job_Grade_Fiscal_Year,AnnualMinimum,AnnualFirstThird,AnnualMidPoint,AnnualSecondThird,AnnualMax,Starting_Full_Time_Salary,Career_Salary_Growth,Percent_Career_Salary_Growth,Number_of_Sal_Adjustments,Number_of_Reclass,REPORTS_TO_PENN_ID,REPORTS_TO_DATE,Manager_Years_Managed,Manager_Months_Managed,Manager_Original_Hire_Date,Manager_Current_Age,Manager_Age_At_Termination,Manager_Edu_Level,Manager_Edu_Level_Desc,Manager_Edu_Level_Year,Manager_YEARS_OF_SERVICE,Manager_Sex_Desc,Manager_Sex,Manager_Race_Desc,Manager_OfColor,Manager_Race,Manager_Job_Title,Termination_Fiscal_Year,TERMINATION_DATE,EMPLOYMENT_STATUS,SEPARATION_REASON,SEPERATION_REASON_DESC,VOLUNTARY,JOBS_ACTIVE,Retirement_Eligible_Date,OVR_RATING_CD_2010,OVR_RATING_DESCR_2010,OVR_RATING_CD_2011,OVR_RATING_DESCR_2011,OVR_RATING_CD_2012,OVR_RATING_DESCR_2012,OVR_RATING_CD_2013,OVR_RATING_DESCR_2013,OVR_RATING_CD_2014,OVR_RATING_DESCR_2014,OVR_RATING_CD_2015,OVR_RATING_DESCR_2015,OVR_RATING_CD_2016,OVR_RATING_DESCR_2016,OVR_RATING_CD_2017,OVR_RATING_DESCR_2017,OVR_RATING_CD_2018,OVR_RATING_DESCR_2018,OVR_RATING_CD_2019,OVR_RATING_DESCR_2019
9021,9022,1986-02-25,R,Asian,1,M,Male,S,NaN,33,H,Master's Degree,2014,TW,19143-2069,PHILADELPHIA,IM,5/1/2029 0:00,2014-09-01,5,61,6,Active,5172,Online Learning,ONLINE LEARNING,51,School of Dental Medicine,Dental,2013-09-12,120391,96056.0,28,DIRECTOR B,0,M1,N,2019.0,58527.0,76085.0,84864.0,93643.0,111201.0,65000.0,31056.0,0.4777,0,1,43662686,12/3/2018,0.0,9.0,9/1/2006,52.0,NaN,K,Doctor of Philosophy (Ph.D.),1998,13.0,Male,M,Caucasian,0.0,C,PROFESSOR C-E,NaN,NaN,A,,NaN,NaN,1,00:00.0,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,R,Meets and frequently exceeds,E,Exceeds,E,Exceeds,R,Meets and frequently exceeds,E,Exceeds
9270,9271,1990-05-31,C,Caucasian,0,F,Female,S,NaN,29,G,Bachelor's Degree,2012,,19125-1310,PHILADELPHIA,,NaN,2016-05-16,3,41,7,Active,5172,Online Learning,ONLINE LEARNING,51,School of Dental Medicine,Dental,2012-08-17,365097,51646.0,27,ONLINE TRAINING SPEC,0,M1,N,2019.0,49686.0,62936.0,69561.0,76185.0,89435.0,30000.0,21646.0,0.7215,2,2,22497341,12/3/2018,0.0,10.0,9/12/2013,33.0,NaN,H,Master's Degree,2014,6.0,Male,M,Asian,1.0,R,DIRECTOR B,NaN,NaN,A,,NaN,NaN,1,00:00.0,X,No Review,X,No Review,X,No Review,M,Meets,M,Meets,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,E,Exceeds
9271,9272,1992-11-24,R,Asian,1,F,Female,M,NaN,26,G,Bachelor's Degree,2015,TW,19107-1877,PHILADELPHIA,H1,8/1/2019 0:00,2015-09-01,4,49,4,Active,5172,Online Learning,ONLINE LEARNING,51,School of Dental Medicine,Dental,2015-09-01,365097,53988.0,27,ONLINE TRAINING SPEC,0,M1,N,2019.0,49686.0,62936.0,69561.0,76185.0,89435.0,47500.0,6488.0,0.1365,1,0,22497341,12/3/2018,0.0,10.0,9/12/2013,33.0,NaN,H,Master's Degree,2014,6.0,Male,M,Asian,1.0,R,DIRECTOR B,NaN,NaN,A,,NaN,NaN,1,00:00.0,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,R,Meets and frequently exceeds,E,Exceeds,E,Exceeds,R,Meets and frequently exceeds


In [86]:
dataset[dataset["Dept_Name"] == "Online Learning"]

,Worker_ID,Birth_Date,RACE,Race_Desc,Race_OfColor,SEX,Sex_Desc,MARITAL_STATUS,AGE_AT_TERMINATION,CURRENT_AGE,EDUCATION_LEVEL,EDUCATION_LEVEL_DESC,EDUCATION_LEVEL_YEAR,COUNTRY_OF_CITIZENSHIP,HOME_ZIP,HOME_CITY,VISA,VISA_EXPIRATION_DATE,PRIMARY_APPT_BEGIN_DATE,YEARS_IN_PRIMARY_JOB,MONTHS_IN_PRIMARY_JOB,YEARS_OF_SERVICE,EMPLOYMENT_STATUS_DESC,HOME_DEPT_ORG,Dept_Name,Dept_Name_Short,HOME_SCHOOL_CTR,School_Name,School_Abbrev,ORIGINAL_HIRE_DATE,PRIMARY_APPT_JOB_CLASS,PRIMARY_APPT_SALARY,JOB_GRADE,JOB_TITLE,IN_UNION,PAY_CYCLE,GRANT_PAID,Job_Grade_Fiscal_Year,AnnualMinimum,AnnualFirstThird,AnnualMidPoint,AnnualSecondThird,AnnualMax,Starting_Full_Time_Salary,Career_Salary_Growth,Percent_Career_Salary_Growth,Number_of_Sal_Adjustments,Number_of_Reclass,REPORTS_TO_PENN_ID,REPORTS_TO_DATE,Manager_Years_Managed,Manager_Months_Managed,Manager_Original_Hire_Date,Manager_Current_Age,Manager_Age_At_Termination,Manager_Edu_Level,Manager_Edu_Level_Desc,Manager_Edu_Level_Year,Manager_YEARS_OF_SERVICE,Manager_Sex_Desc,Manager_Sex,Manager_Race_Desc,Manager_OfColor,Manager_Race,Manager_Job_Title,Termination_Fiscal_Year,TERMINATION_DATE,EMPLOYMENT_STATUS,SEPARATION_REASON,SEPERATION_REASON_DESC,VOLUNTARY,JOBS_ACTIVE,Retirement_Eligible_Date,OVR_RATING_CD_2010,OVR_RATING_DESCR_2010,OVR_RATING_CD_2011,OVR_RATING_DESCR_2011,OVR_RATING_CD_2012,OVR_RATING_DESCR_2012,OVR_RATING_CD_2013,OVR_RATING_DESCR_2013,OVR_RATING_CD_2014,OVR_RATING_DESCR_2014,OVR_RATING_CD_2015,OVR_RATING_DESCR_2015,OVR_RATING_CD_2016,OVR_RATING_DESCR_2016,OVR_RATING_CD_2017,OVR_RATING_DESCR_2017,OVR_RATING_CD_2018,OVR_RATING_DESCR_2018,OVR_RATING_CD_2019,OVR_RATING_DESCR_2019
652,653,1982-09-08,C,Caucasian,0,F,Female,S,NaN,37,H,Master's Degree,2015,,19143-2051,PHILADELPHIA,,NaN,2011-09-19,8,97,8,Active,297,Online Learning,ONLINE LEARNING,2,School of Arts and Sciences,SAS,2011-09-19,120391,81500.00,28,DIRECTOR B,0,M1,N,2019.0,58527.0,76085.0,84864.0,93643.0,111201.0,45000.0,36500.00,0.8111,0,3,20915978,12/3/2018,0.0,10.0,3/25/2011,58.0,NaN,K,Doctor of Philosophy (Ph.D.),2008,8.0,Female,F,Caucasian,0.0,C,DIRECTOR D,NaN,NaN,A,,NaN,NaN,1,00:00.0,X,No Review,X,No Review,X,No Review,R,Meets and frequently exceeds,E,Exceeds,R,Meets and frequently exceeds,E,Exceeds,X,No Review,0,No Overall Rating,X,No Review
653,654,1985-05-26,C,Caucasian,0,M,Male,M,31.0,34,K,Doctor of Philosophy (Ph.D.),2013,,15211-1848,PITTSBURGH,,NaN,2013-04-27,3,40,3,Seperated or teminated,297,Online Learning,ONLINE LEARNING,2,School of Arts and Sciences,SAS,2013-04-25,120417,73024.42,28,ASSOC DIRECTOR D,0,M1,N,2017.0,56253.0,73129.0,81567.0,90005.0,106881.0,55000.0,18024.42,0.3277,0,1,20915978,3/25/2014,2.0,29.0,3/25/2011,58.0,55.0,K,Doctor of Philosophy (Ph.D.),2008,5.0,Female,F,Caucasian,0.0,C,DIRECTOR D,2017.0,8/2/2016 0:00,T,41,Accepted New Job (Non-Academic),1.0,0,00:00.0,X,No Review,X,No Review,X,No Review,X,No Review,E,Exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,X,No Review,X,No Review,X,No Review
654,655,1982-08-23,C,Caucasian,0,M,Male,M,30.0,37,H,Master's Degree,2011,,19143,PHILADELPHIA,,NaN,2011-06-14,1,20,1,Seperated or teminated,297,Online Learning,ONLINE LEARNING,2,School of Arts and Sciences,SAS,2011-06-14,120602,56000.00,27,ASSOC DIRECTOR C,0,M1,N,2013.0,44336.0,56159.0,62071.0,67982.0,79805.0,46000.0,10000.00,0.2173,0,1,20915978,5/3/2012,0.0,9.0,3/25/2011,58.0,51.0,K,Doctor of Philosophy (Ph.D.),2008,1.0,Female,F,Caucasian,0.0,C,DIRECTOR D,2013.0,2/7/2013 0:00,T,41,Accepted New Job (Non-Academic),1.0,0,00:00.0,X,No Review,X,No Review,E,Exceeds,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review
655,656,1986-02-03,C,Caucasian,0,M,Male,S,NaN,33,G,Bachelor's Degree,2008,,19125-2903,PHILADELPHIA,,NaN,2017-05-17,2,29,2,Active,297,Online Learning,ONLINE LEARNING,2,School of Arts and Sciences,SAS,2017-05-17,120602,61800.00,27,ASSOC DIRECTOR C,0,M1,N,2019.0,49686.0,62936.0,69561.0,76185.0,89435.0,60000.0,1800.00,0.0300,0,0,20915978,12/3/2018,0.0,10.0,3/25/

In [87]:
dataset[dataset["HOME_DEPT_ORG"] == 8317]

,Worker_ID,Birth_Date,RACE,Race_Desc,Race_OfColor,SEX,Sex_Desc,MARITAL_STATUS,AGE_AT_TERMINATION,CURRENT_AGE,EDUCATION_LEVEL,EDUCATION_LEVEL_DESC,EDUCATION_LEVEL_YEAR,COUNTRY_OF_CITIZENSHIP,HOME_ZIP,HOME_CITY,VISA,VISA_EXPIRATION_DATE,PRIMARY_APPT_BEGIN_DATE,YEARS_IN_PRIMARY_JOB,MONTHS_IN_PRIMARY_JOB,YEARS_OF_SERVICE,EMPLOYMENT_STATUS_DESC,HOME_DEPT_ORG,Dept_Name,Dept_Name_Short,HOME_SCHOOL_CTR,School_Name,School_Abbrev,ORIGINAL_HIRE_DATE,PRIMARY_APPT_JOB_CLASS,PRIMARY_APPT_SALARY,JOB_GRADE,JOB_TITLE,IN_UNION,PAY_CYCLE,GRANT_PAID,Job_Grade_Fiscal_Year,AnnualMinimum,AnnualFirstThird,AnnualMidPoint,AnnualSecondThird,AnnualMax,Starting_Full_Time_Salary,Career_Salary_Growth,Percent_Career_Salary_Growth,Number_of_Sal_Adjustments,Number_of_Reclass,REPORTS_TO_PENN_ID,REPORTS_TO_DATE,Manager_Years_Managed,Manager_Months_Managed,Manager_Original_Hire_Date,Manager_Current_Age,Manager_Age_At_Termination,Manager_Edu_Level,Manager_Edu_Level_Desc,Manager_Edu_Level_Year,Manager_YEARS_OF_SERVICE,Manager_Sex_Desc,Manager_Sex,Manager_Race_Desc,Manager_OfColor,Manager_Race,Manager_Job_Title,Termination_Fiscal_Year,TERMINATION_DATE,EMPLOYMENT_STATUS,SEPARATION_REASON,SEPERATION_REASON_DESC,VOLUNTARY,JOBS_ACTIVE,Retirement_Eligible_Date,OVR_RATING_CD_2010,OVR_RATING_DESCR_2010,OVR_RATING_CD_2011,OVR_RATING_DESCR_2011,OVR_RATING_CD_2012,OVR_RATING_DESCR_2012,OVR_RATING_CD_2013,OVR_RATING_DESCR_2013,OVR_RATING_CD_2014,OVR_RATING_DESCR_2014,OVR_RATING_CD_2015,OVR_RATING_DESCR_2015,OVR_RATING_CD_2016,OVR_RATING_DESCR_2016,OVR_RATING_CD_2017,OVR_RATING_DESCR_2017,OVR_RATING_CD_2018,OVR_RATING_DESCR_2018,OVR_RATING_CD_2019,OVR_RATING_DESCR_2019
11401,11402,1972-11-16,B,African American,1,F,Female,S,44.0,46,,NaN,,,08046-1303,WILLINGBORO,,NaN,2001-11-01,15,182,15,Seperated or teminated,8317,Arthur Ross Gallery,ART ROSS GALL,83,Provost's Center,Provost Center,2001-11-01,330024,65395.0,27,ASSOCIATE CURATOR,0,M1,N,2017.0,47756.0,60491.0,66858.0,73225.0,85960.0,32000.0,33395.0,1.0435,0,1,30235092,5/21/2012,4.0,56.0,2/29/2008,69.0,66.0,H,Master's Degree,1976,8.0,Female,F,Caucasian,0.0,C,DIRECTOR C,2017.0,1/31/2017 0:00,T,41,Accepted New Job (Non-Academic),1.0,0,00:00.0,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,X,No Review,X,No Review,X,No Review


In [88]:
dataset[dataset["Dept_Name"] == "Arthur Ross Gallery"]

,Worker_ID,Birth_Date,RACE,Race_Desc,Race_OfColor,SEX,Sex_Desc,MARITAL_STATUS,AGE_AT_TERMINATION,CURRENT_AGE,EDUCATION_LEVEL,EDUCATION_LEVEL_DESC,EDUCATION_LEVEL_YEAR,COUNTRY_OF_CITIZENSHIP,HOME_ZIP,HOME_CITY,VISA,VISA_EXPIRATION_DATE,PRIMARY_APPT_BEGIN_DATE,YEARS_IN_PRIMARY_JOB,MONTHS_IN_PRIMARY_JOB,YEARS_OF_SERVICE,EMPLOYMENT_STATUS_DESC,HOME_DEPT_ORG,Dept_Name,Dept_Name_Short,HOME_SCHOOL_CTR,School_Name,School_Abbrev,ORIGINAL_HIRE_DATE,PRIMARY_APPT_JOB_CLASS,PRIMARY_APPT_SALARY,JOB_GRADE,JOB_TITLE,IN_UNION,PAY_CYCLE,GRANT_PAID,Job_Grade_Fiscal_Year,AnnualMinimum,AnnualFirstThird,AnnualMidPoint,AnnualSecondThird,AnnualMax,Starting_Full_Time_Salary,Career_Salary_Growth,Percent_Career_Salary_Growth,Number_of_Sal_Adjustments,Number_of_Reclass,REPORTS_TO_PENN_ID,REPORTS_TO_DATE,Manager_Years_Managed,Manager_Months_Managed,Manager_Original_Hire_Date,Manager_Current_Age,Manager_Age_At_Termination,Manager_Edu_Level,Manager_Edu_Level_Desc,Manager_Edu_Level_Year,Manager_YEARS_OF_SERVICE,Manager_Sex_Desc,Manager_Sex,Manager_Race_Desc,Manager_OfColor,Manager_Race,Manager_Job_Title,Termination_Fiscal_Year,TERMINATION_DATE,EMPLOYMENT_STATUS,SEPARATION_REASON,SEPERATION_REASON_DESC,VOLUNTARY,JOBS_ACTIVE,Retirement_Eligible_Date,OVR_RATING_CD_2010,OVR_RATING_DESCR_2010,OVR_RATING_CD_2011,OVR_RATING_DESCR_2011,OVR_RATING_CD_2012,OVR_RATING_DESCR_2012,OVR_RATING_CD_2013,OVR_RATING_DESCR_2013,OVR_RATING_CD_2014,OVR_RATING_DESCR_2014,OVR_RATING_CD_2015,OVR_RATING_DESCR_2015,OVR_RATING_CD_2016,OVR_RATING_DESCR_2016,OVR_RATING_CD_2017,OVR_RATING_DESCR_2017,OVR_RATING_CD_2018,OVR_RATING_DESCR_2018,OVR_RATING_CD_2019,OVR_RATING_DESCR_2019
9474,9475,1950-04-29,C,Caucasian,0,F,Female,S,NaN,69,H,Master's Degree,1976,,19096-3102,WYNNEWOOD,,NaN,2008-02-29,11,140,11,Active,444,Arthur Ross Gallery,ARTHUR ROSS GALLERY,4,Provost Interdisciplinary Programs,Provost,2008-02-29,120392,122630.0,29,DIRECTOR C,0,M1,N,2019.0,71403.0,92824.0,103534.0,114244.0,135665.0,72000.0,50630.0,0.7031,0,1,10108503,12/4/2018,0.0,10.0,7/1/1998,66.0,NaN,J,"Other doctorate (Dr of Educ., Dr of Sct., LL.D.)",1983,21.0,Female,F,African American,1.0,B,PROFESSOR A,NaN,NaN,A,,NaN,NaN,1,00:00.0,E,Exceeds,E,Exceeds,0,No Overall Rating,0,No Overall Rating,0,No Overall Rating,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds,E,Exceeds
11400,11401,1993-12-02,C,Caucasian,0,F,Female,S,25.0,25,,NaN,,,19143-3503,PHILADELPHIA,,NaN,2017-09-11,1,20,5,Seperated or teminated,444,Arthur Ross Gallery,ARTHUR ROSS GALLERY,4,Provost Interdisciplinary Programs,Provost,2013-09-16,525625,35550.0,24,ADMIN ASST B,0,W1,N,2019.0,31360.0,38678.0,42337.0,45995.0,53313.0,3000.0,32550.0,10.8500,0,0,30235092,12/17/2018,0.0,5.0,2/29/2008,69.0,69.0,H,Master's Degree,1976,11.0,Female,F,Caucasian,0.0,C,DIRECTOR C,2019.0,5/23/2019 0:00,T,41,Accepted New Job (Non-Academic),1.0,0,00:00.0,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,X,No Review,M,Meets
11401,11402,1972-11-16,B,African American,1,F,Female,S,44.0,46,,NaN,,,08046-1303,WILLINGBORO,,NaN,2001-11-01,15,182,15,Seperated or teminated,8317,Arthur Ross Gallery,ART ROSS GALL,83,Provost's Center,Provost Center,2001-11-01,330024,65395.0,27,ASSOCIATE CURATOR,0,M1,N,2017.0,47756.0,60491.0,66858.0,73225.0,85960.0,32000.0,33395.0,1.0435,0,1,30235092,5/21/2012,4.0,56.0,2/29/2008,69.0,66.0,H,Master's Degree,1976,8.0,Female,F,Caucasian,0.0,C,DIRECTOR C,2017.0,1/31/2017 0:00,T,41,Accepted New Job (Non-Academic),1.0,0,00:00.0,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,R,Meets and frequently exceeds,X,No Review,X,No Review,X,No Review
11402,11403,1979-07-08,C,Caucasian,0,F,Female,S,NaN,40,H,Master's Degree,2004,,19148-1325,PHILADELPHIA,,NaN,2017-03-06,2,31,8,Active,444,Arthur Ross Gallery,ARTHUR ROSS GALLERY,4,Provost Interdisciplinary Programs,Provost,2011-09-06,330024,61830.0,27,ASSOCIATE CURATOR,0

## HOME_SCHOOL_CTR

In [95]:
print(np.sort(dataset["HOME_SCHOOL_CTR"].unique()))

[ 2  4  6  7 13 19 24 26 32 33 35 36 40 50 51 56 58 60 61 62 78 79 81 83
 84 85 86 87 90 91 92 93 95 96 98]


In [99]:
print(len(np.sort(dataset["HOME_SCHOOL_CTR"].unique())))

35


In [96]:
print(dataset["HOME_SCHOOL_CTR"].value_counts().head())

40    6971
96    1364
2     1284
7     1257
58     985
Name: HOME_SCHOOL_CTR, dtype: int64


In [97]:
print(dataset["HOME_SCHOOL_CTR"].value_counts().tail())

78    48
98    45
61    41
19    34
84     3
Name: HOME_SCHOOL_CTR, dtype: int64


In [98]:
df3 = dataset[['HOME_SCHOOL_CTR','School_Name']]
check_for_discrepency(df3)

All good


,HOME_SCHOOL_CTR,School_Name


In [106]:
# df[['HOME_SCHOOL_CTR', 'School_Name']].where(df.Cond == "X", df[['HOME_SCHOOL_CTR', 'School_Name']].values)
dataset[dataset["HOME_SCHOOL_CTR"] == 58]["School_Name"]

361      School of Veterinary Medicine
362      School of Veterinary Medicine
1108     School of Veterinary Medicine
1121     School of Veterinary Medicine
1122     School of Veterinary Medicine
                     ...              
17260    School of Veterinary Medicine
17261    School of Veterinary Medicine
17262    School of Veterinary Medicine
17263    School of Veterinary Medicine
17402    School of Veterinary Medicine
Name: School_Name, Length: 985, dtype: object

## ORIGINAL_HIRE_DATE

In [72]:
dataset["ORIGINAL_HIRE_DATE"] = pd.DatetimeIndex(dataset["ORIGINAL_HIRE_DATE"])

In [74]:
print(dataset["ORIGINAL_HIRE_DATE"])

0       2008-09-08
1       2007-07-27
2       2004-12-03
3       1987-02-16
4       2011-09-06
           ...    
19366   1998-05-11
19367   2014-06-15
19368   2011-06-08
19369   2000-11-13
19370   2009-11-11
Name: ORIGINAL_HIRE_DATE, Length: 19371, dtype: datetime64[ns]


In [75]:
types = []
months = []
days = []
years = []

for date in dataset["ORIGINAL_HIRE_DATE"]:
    types.append(type(date))
    months.append(date.month)
    days.append(date.day)
    years.append(date.year)

typesdf = pd.DataFrame(types)
monthsdf = pd.DataFrame(months)
daysdf = pd.DataFrame(days)
yearsdf = pd.DataFrame(years)

In [77]:
print("Months: Value Counts-")
print(monthsdf[0].value_counts())
print("Months: Unique Values-")
print(np.sort(monthsdf[0].unique()))
print("Years: Value Counts-")
print(yearsdf[0].value_counts())
print("Years: Unique Values-")
print(np.sort(yearsdf[0].unique()))
print("Days: Value Counts-")
print(daysdf[0].value_counts())
print("Days: Unique Values-")
print(np.sort(daysdf[0].unique()))


Months: Value Counts-
9     2592
8     1953
6     1919
7     1917
1     1726
10    1671
5     1489
11    1402
4     1292
3     1290
2     1265
12     855
Name: 0, dtype: int64
Months: Unique Values-
[ 1  2  3  4  5  6  7  8  9 10 11 12]
Years: Value Counts-
2016    1286
2017    1260
2018    1179
2015    1117
2014    1065
        ... 
1955       1
1950       1
1954       1
1943       1
1960       1
Name: 0, Length: 68, dtype: int64
Years: Unique Values-
[1943 1950 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965
 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979
 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007
 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Days: Value Counts-
1     2551
2      833
15     786
3      781
5      778
4      692
14     671
8      653
6      628
7      602
11     596
13     592
9      587
12     578
10     577
16     565
1